In [28]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer 

In [4]:
# Get the absolute path to the data directory
abs_dir = os.path.abspath(os.path.dirname(os.getcwd()))

In [5]:
df = pd.read_csv(os.path.join(abs_dir, 'scripts/data/cbe_reviews_20250608_162756.csv'))

In [6]:
pd.set_option('display.max_colwidth', None)

df.head(5)

,review_text,rating,upvote,date,bank_name,source
0,really am happy to this app it is Siple to use everything,5,0,2025-06-07,Commercial Bank of Ethiopia,Google Play
1,I liked this app. But the User interface is very basic and not attractive at all.,2,0,2025-06-07,Commercial Bank of Ethiopia,Google Play
2,"""Why don’t your ATMs support account-to-account transfers like other countries( Kenya, Nigeria , South africa)""",4,0,2025-06-06,Commercial Bank of Ethiopia,Google Play
3,what is this app problem???,1,0,2025-06-05,Commercial Bank of Ethiopia,Google Play
4,the app is proactive and a good connections.,5,0,2025-06-05,Commercial Bank of Ethiopia,Google Play


In [7]:
# check types 
df.dtypes

review_text    object
rating          int64
upvote          int64
date           object
bank_name      object
source         object
dtype: object

In [8]:
df.describe()

,rating,upvote
count,4000.000000,4000.000000
mean,4.110500,8.488250
std,1.474562,82.261194
min,1.000000,0.000000
25%,4.000000,0.000000
50%,5.000000,1.000000
75%,5.000000,1.000000
max,5.000000,3025.000000


In [9]:
# check for null values in the DataFrame
df.isnull().sum()

review_text    0
rating         0
upvote         0
date           0
bank_name      0
source         0
dtype: int64

### Remove Duplicate Entries

When I check for duplicate entries in general, I find 81 duplicate rows. However, when I define duplicates based on both review_text and rating, the number increases to 1,162 rows. This indicates that the most influential factors in identifying duplicates are the review text and the rating.

In [10]:
# Check for duplicated data
dublicated_data = df.duplicated(subset=['review_text', 'rating'], keep=False).sum()
print(f'data duplicated: {dublicated_data}')
df[df.duplicated()]

data duplicated: 1162


,review_text,rating,upvote,date,bank_name,source
15,good,5,0,2025-06-04,Commercial Bank of Ethiopia,Google Play
74,good,5,0,2025-05-23,Commercial Bank of Ethiopia,Google Play
76,good,5,0,2025-05-23,Commercial Bank of Ethiopia,Google Play
86,ok,5,0,2025-05-22,Commercial Bank of Ethiopia,Google Play
103,best,5,0,2025-05-21,Commercial Bank of Ethiopia,Google Play
...,...,...,...,...,...,...
3381,Good,5,1,2024-02-11,Commercial Bank of Ethiopia,Google Play
3396,Good,5,1,2024-02-11,Commercial Bank of Ethiopia,Google Play
3409,Good,5,1,2024-02-09,Commercial Bank of Ethiopia,Google Play
3641,Good,5,1,2024-01-05,Commercial Bank of Ethiopia,Google Play


In [11]:
# remove duplicated data
df.drop_duplicates(subset=['review_text', 'rating'], keep='first', inplace=True)

In [12]:
# change date format from object to 'YYYY-MM-DD' format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df.dtypes

review_text            object
rating                  int64
upvote                  int64
date           datetime64[ns]
bank_name              object
source                 object
dtype: object

In [13]:
df.shape

(2988, 6)

In [17]:
!pip install transformers #distilBERT-base-uncased, which helps to classify the reviews
!pip install huggingface_hub[hf_xet] # for downloading the model from Hugging Face, for the purpose of sentiment analysis


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\elsha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#distilBERT-base-uncased,': Expected package name at the start of dependency specifier
    #distilBERT-base-uncased,
    ^


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\elsha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [16]:
from transformers import pipeline

# Load the sentiment analysis pipeline,  creates a pipeline for a specific NLP task. In this case, the task is sentiment analysis.
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english") 
#pipline is used to perform sentiment analysis on the review_text column

# Apply sentiment to review_text column
df['sentiment_result'] = df['review_text'].astype(str).apply(lambda x: sentiment_pipeline(x)[0]) 
#the above line applies the sentiment analysis pipeline to each review text and stores the result in a new column 'sentiment_result'
#astype(str) ensures that the review_text is treated as a string, even if it contains null values or other types.
#sentiment_pipeline(x)[0] returns the first result of the sentiment analysis, which is a dictionary containing the sentiment label and score.

df['sentiment_label'] = df['sentiment_result'].apply(lambda x: x['label'])
#the above line extracts the sentiment label (e.g., 'POSITIVE' or 'NEGATIVE') from the sentiment result and stores it in a new column 'sentiment_label'
#apply(lambda x: x['label']) is used to extract the label from each sentiment result.

df['sentiment_score'] = df['sentiment_result'].apply(lambda x: x['score'])
#the above line extracts the sentiment score (a numerical value indicating the confidence of the sentiment) from the sentiment result and stores it in a new column 'sentiment_score'

print(df[['review_text', 'sentiment_label', 'sentiment_score',"sentiment_result"]].head(5))


Device set to use cpu


                                                                                                       review_text  \
0                                                        really am happy to this app it is Siple to use everything   
1                                I liked this app. But the User interface is very basic and not attractive at all.   
2  "Why don’t your ATMs support account-to-account transfers like other countries( Kenya, Nigeria , South africa)"   
3                                                                                      what is this app problem???   
4                                                                     the app is proactive and a good connections.   

  sentiment_label  sentiment_score  \
0        POSITIVE         0.998870   
1        NEGATIVE         0.999684   
2        NEGATIVE         0.996465   
3        NEGATIVE         0.999623   
4        POSITIVE         0.999868   

                                     sentiment_result  
0  {'l

In [23]:
agg_mean = df.groupby(["bank_name", "rating"])["sentiment_score"].mean().reset_index()
#this line groups the DataFrame by 'bank_name' and 'rating', calculates the mean sentiment score for each group, and resets the index to create a new DataFrame with the results.
#reset_index() is used to convert the grouped DataFrame back into a regular DataFrame format.

print(arg_mean.head(5))

                     bank_name  rating  sentiment_score
0  Commercial Bank of Ethiopia       1         0.976435
1  Commercial Bank of Ethiopia       2         0.963838
2  Commercial Bank of Ethiopia       3         0.959347
3  Commercial Bank of Ethiopia       4         0.953669
4  Commercial Bank of Ethiopia       5         0.960054


In [27]:
# Rename the column for clarity (optional)
agg_mean.rename(columns={'sentiment_score': 'avg_sentiment_score'}, inplace=True)
#this line renames the 'sentiment_score' column to 'avg_sentiment_score' for clarity, indicating that it represents the average sentiment score for each group.

# agg_df = agg_mean.rename(columns={'sentiment_score': 'average_sentiment_score'}, inplace=True)
print(arg_mean.head(5))

                     bank_name  rating  sentiment_score
0  Commercial Bank of Ethiopia       1         0.976435
1  Commercial Bank of Ethiopia       2         0.963838
2  Commercial Bank of Ethiopia       3         0.959347
3  Commercial Bank of Ethiopia       4         0.953669
4  Commercial Bank of Ethiopia       5         0.960054


## Thematic Analysis
A theme refers to a recurring concept or topic within user reviews. For this challenge, themes will help summarize user feedback into actionable categories for the banks.

### Keyword Extraction & Manual/Rule-Based Clustering

In [35]:
verctorizer = TfidfVectorizer(max_features=100)

# Fit and transform the review_text column to create a TF-IDF matrix

tfidf_verctorized = verctorizer.fit_transform(df['review_text'].astype(str))

keywords = verctorizer.get_feature_names_out()

print(f"Number of keywords: {len(keywords)}")

print(f"Keywords: {keywords[:20]}")  # Display the first 10 keywords


Number of keywords: 100
Keywords: ['account' 'after' 'all' 'am' 'amazing' 'an' 'and' 'app' 'application'
 'apps' 'are' 'as' 'at' 'bank' 'banking' 'be' 'best' 'but' 'by' 'can']
